---

# CSCI 3202, Fall 2022
# Homework 4
# Due: Friday September 30, 2022 at 6:00 PM

<br> 

### Your name: Giuliano Costa

<br> 

---

Some useful packages and libraries:




In [149]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter


---

# Hidden Markov Models
In this assignment, you will build a Hidden Markov Model from a data set to detect typos in text. One major difference between this assignment and previous assignments in this class is we're not providing any skeleton code, you need to generate your code from scratch. You can use any code provided during lecture or on previous assignments. 
<p><strong>Please read the entire assignment before beginning.</strong> 
    
<p>The main components of the assignment are the following:

### Part I - Building an HMM from data
1.  Implement a method to parse the data file and build an HMM from the data.
2.  Implement a method, Laplace smoothing, to smooth the transition and emission probabilities

### Part II - Implement the Viterbi algorithm
1.	Implement the Viterbi algorithm for finding the most likely sequence of states through the HMM, given "evidence"; and
2.	Run your code on a separate dataset and explore its performance.
    
### Part III - Summarize your results
    



## Part 1 - Building an HMM from data
The first part of the assignment is to build an HMM from data. Recall that an HMM involves a hidden state that changes over time, as well as observable evidence that is used to infer the hidden state. In our example from lecture, we used the observation of how a patient felt to infer whether they were healthy or sick. 

An HMM is defined by three sets of probabilities:
1.	For each state s, the probability of observing each output o at that state (P(E[t]=o | X[t]=s). These are the emission probabilities.
2.	From each state s, the probability of traversing to every other state s' in one time step (P(X[t+1]=s' | X[t]=s)). These are the transition probabilities.
3.	A distribution over the start state (P(X[0])). This is the initial state distribution.

For the start state distribution (P(X[0])), in this assignment, you will assume that there is a single dummy start state, distinct from all other states, and to which the HMM can never return.  When you implement the Viterbi algorithm in Part II, you will need to include the probability of making a transition from this dummy start state to each of the other states.

### Generate the conditional probability tables
To generate the transition and emission probability tables, use the data file called <em>typos20.data</em> on Canvas that contains sequences of state-output pairs, i.e., sequences of the form: <br>

x[1] e[1] <br>
x[2] e[2] <br>
. <br>
. <br>
. <br>
x[n] e[n]. <br>  

In this data, the x[i] is the state and the e[i] is the observation at time i. 

For instance, to estimate the probability of output o being observed in state s, you count the number of times that output o appears with state s in the given data, and divide by a normalization constant (so that the probabilities of all outputs from that state add up to one).  In this case, that normalization constant would simply be the number of times that state s appears at all in the data.

In this problem, state is letter that should have been typed, and output is the letter that was typed.  Given a sequence of outputs (i.e., actually typed letters), the problem is to reconstruct the hidden state sequence (i.e., the intended sequence of letters).  Thus, data for this problem looks like this:

i i <br>
n n <br>
t t <br>
r r <br>
o o <br>
d x <br>
u u <br>
c c <br>
t t <br>
i i <br>
o i <br>
n n <br>
_ _ <br>
t t <br>
h h <br>
e e <br>
_ _ <br>

where the left column is the correct text and the right column contains text with errors.

### Data
Data for this assignment was generated by starting with a text document, in this case, the Unabomber's Manifesto, which was chosen not for political reasons, but for its convenience being available on-line and of about the right lens, converting all numbers and punctuation to white space and converting all letters to lower case.  The remaining text is a sequence only over the lower case letters and the space character, represented in the data files by an underscore character.  Next, typos were artificially added to the data as follows: with 80% probability, the correct letter is transcribed, but with 20% probability, a randomly chosen neighbor (on an ordinary physical keyboard) of the letter is transcribed instead.  Space characters are always transcribed correctly. 

As an example, the original document begins:

<em>introduction the industrial revolution and its consequences have been a disaster for the human race they have greatly increased the life expectancy of those of us who live in advanced countries but they have destabilized society have made life unfulfilling have subjected human beings to indignities have led to widespread psychological suffering in the third world to physical suffering as well and have inflicted severe damage on the natural world the continued development of technology will worsen the situation it will certainly subject human beings to greater indignities and inflict greater damage on the natural world it will probably lead to greater social disruption and psychological suffering and it may lead to increased physical suffering even in advanced countries the industrial technological system may survive or it may break down if it survives it may eventually achieve a low level of physical and psychological suffering but only after passing through a long and very painful period of adjustment and only at the cost of permanently reducing human beings and many other living organisms to engineered products and mere cogs in the social machine </em>

With 20% noise, it looks like this:

<em>introductipn the industfial revolhtjon and its consequences bafw newn a diszster rkr the yumab race thdy have grwatky increased the ljte esoectandy od thosr of is who libe in advanced coubfries but they have fewtabipuzee xociwty have made life ujfuorillkng have wubjwdted humah beints to incihbjtids have led to qidespreze lsyxhllotical shffeding kn tne third wkrld to phyxicql sufcefimg as weol and hqve ingoidtex srvere damsge on the natural world the confinued developmeng of twvhjllogy will wotsen thd situation it wull certaknly sunjrct yyman beingw tl greater ibdignities snd infpixt greagwr damsge on fhe natural alrld it wjlk probably lwad tk grezter sofiqp disrupgiln and pstchokofucal wufterkng anc it may kead fl uncreqxed pgusiczl sucfreinh even in acgajved countries the indhsteial tedhnologicak system may survivr or ut nay brezk down uf it survives it nay evenyuakly achieve a los lwvel of phyxkcal and psycyoligical sufveribg but only after passing theough a long amd very painful periox od adjuwtmebt and only at the fost kf permsnently reducing hymaj veings abs nsjy otgwr kuving orbanisms to envineered leoduxfs amd mere clgs in thr soxiap maxhjne </em>

The error rate (fraction of characters that are mistyped) is about 16.5% (less than 20% because space characters were not corrupted).

### Implement Laplace smoothing
When you generate the transition and emission probabilities, you will also need to smooth the estimates.  Here's the rationale, consider flipping a coin for which the probability of heads is p, where p is unknown, and our goal is to estimate p. We count how many times the coin comes up heads and divide by the total number of coin flips.  If we flip the coin 1000 times and it comes up heads 367 times, it is very reasonable to estimate p as approximately 0.367.  However, suppose we flip the coin only twice and we get heads both times.  Is it reasonable to estimate p as 1.0?  Intuitively, given that we only flipped the coin twice, we don’t have enough data to conclude that the coin will always come up heads. Smoothing is a way of avoiding these types of conclusions when we have limited data available.  

A simple smoothing method, called Laplace smoothing (or Laplace's law of succession or add-one smoothing in your textbook), is to estimate p by:

<p>

$$\frac{(1+number of heads)}{(2+number of flips)}$$

We add 1 to each output that could be observed in the numerator, and add the total number of states to the denominator.

If we are keeping count of the number of heads and the number of tails, this rule is equivalent to starting each of our counts at one, rather than zero.  This latter view generalizes to the case in which there are more than two possible outcomes (for instance, estimating the probability of a die coming up on each of its six faces). 
Another advantage of Laplace smoothing is that it avoids estimating any probabilities to be zero, even for events never observed in the data.  For HMMs, this is important since zero probabilities can be problematic for some algorithms.
    
For this assignment, you should use Laplace-smoothed estimates of probabilities.  For instance, returning to the problem of estimating the probability of output o being observed in state s, you would use one plus the number of times output o appears in state s in the given data, divided by a normalization constant. In this case, the normalization constant would be the number of times state s appears in the data, plus the total number of possible outputs. You will need to also work out Laplace-smoothed estimates for item 2, i.e., for the probability of making a transition from one state to another, as well as the probability of making a transition from the dummy start state to any of the other states. 


In [150]:
## Your code goes here.
#typos = pd.read_csv("./data/typos11_test.data", sep=' ', header = None, names=["State", "Output"])
#array = typos.to_dict()

#letters = ['_', 'a', 'b', 'c', 'd', 'e', 'g', 'f', 'h', 'j', 'k', 'i', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

#typos
## Your code needs to display the first 10 lines of your emission and transition tables
## and generate an output file with the complete tables, called Assignment8Output_Lastname_Firstname

# Your code also needs to be structured as an HMM class

In [151]:

class HMM():
    def __init__(self, typos_df):
        
        #### Initial Distribution ####
        states = typos_df['State']
        state_sums = {} 
        total = 0
        letters = ''.join(sorted(set(states)))
        
        for i in letters:
            state_sums[i] = states.str.count(i).sum()
            total += state_sums[i]
        #Sum state letter counts and divide by sum of letters
        a = {k: v / total for k, v in state_sums.items()}        
        init_distribution = pd.DataFrame.from_dict(a, orient="index", columns=['State'])        
        
        #### Transition Matrix ####
        #S to S'
        # Create a raw transition matrix
        #print("state", typos['State'])
        state = typos_df['State'].to_numpy()
        state_prime = typos_df['State'].to_numpy()
        state_prime = np.roll(state_prime, -1, axis=0)

        a = np.column_stack((state, state_prime))

        matrix = pd.Series(Counter(map(tuple, a))).unstack().fillna(0)
        matrix = matrix.rename_axis("S", axis="rows")
        matrix = matrix.rename_axis("S'", axis="columns")

        # Normalize the rows according to the laplace smoothing.
        denom = matrix
        numerator = matrix + 1
        transition_probs = numerator.divide(denom.sum(axis=1) + 27, axis=0)
        self.trans = transition_probs

        #### Emission ####
        a = typos_df.to_numpy()
        matrix = pd.Series(Counter(map(tuple, a))).unstack().fillna(0)


        matrix = matrix.rename_axis("State", axis="rows")
        matrix = matrix.rename_axis("Output", axis="columns")

        # Normalize the rows according to the laplace smoothing.
        denom = matrix
        numerator = matrix + 1
        emission_probs = numerator.divide(denom.sum(axis=1) + 27, axis=0)
        self.em = emission_probs
        self.init = init_distribution
        self.letters = letters
        
    def tables(self):
        self.em.to_csv('./data/HW4_Costa_Giuliano.data', sep=',', mode='a')
        self.init.to_csv('./data/HW4_Costa_Giuliano.data', sep=',', mode='a')
        self.trans.to_csv('./data/HW4_Costa_Giuliano.data', sep=',', mode='a')
        
        return  self.em, self.init, self.trans


In [152]:
# typos = pd.read_csv("./data/typos20.data", sep=' ', header = None, names=["State", "Output"])

# test = HMM(typos)
# test.trans


## Part II - Viterbi - finding the most likely sequence

Once your CPTs for your HMM are built, the second part of the assignment is to implement the Viterbi algorithm and use your CPTs to compute the most probable sequence of states (according to the HMM that you built from the data) for a given sequence of outputs.

There is another file on Canvas called *typos20Test.data* that contains test sequences of state-output pairs. You will provide your Viterbi code with just the output part of each of these sequences, and from this, you must compute the most likely sequence of states to produce the output sequence. The state part of these sequences is provided so that you can compare the estimated state sequences generated by your code to the actual state sequences that generated this data.  Note that these two sequences will not necessarily be identical, even if you have correctly implemented the Viterbi algorithm.

A numerical tip: the Viterbi algorithm involves multiplying many probabilities together.  Since each of these numbers is smaller than one (possibly much smaller), you can end up working with numbers that are tiny enough to be computationally indistinguishable from zero. To avoid this problem, it is recommended that you work with log probabilities. To do so, rather than multiplying two probabilities p and q, simply add their logarithms using the rule:
<p>
<center> log(pq) = log(p) + log(q) </center>

You will probably find that there is never a need to store or manipulate actual probabilities; instead, everything can be done using log probabilities.
The text reconstructed using an HMM with the Viterbi algorithm looks like this:
    <p>
*introduction the industrial revoluthon and its consequences bare neen a dissster ror the tuman race they have greatly increased the lite esoectandy od those of is who libe in advanced counfries but they have festabupusee cocisty have made live intiorilling have wibjested human beints to incingitids have led to widesprese lsysullotical suffeding in the third world to physical surcefing as weol and have ingoistes severe damage on the natural world the continued developmeng of techillogy will wotsen the situation it will certaknly sunirct tyman beinge tl greater indithities and infoist greager damage on the natural aleld it will probably owad to grester sofial distuption and pstchomofucal wiftering and it may kead fl increqxed ogusical suctreing even in achanved countries the industeial technologicak system may survive or ut nay break down if it survives it nay eventually achieve a los level of physical and psycholigical survering but only arter passing theough a long and very paindul perios od adjustment and only at the fost of permanently reducing human veings ans nany other kiving organisms to envineered leodusts and mere clys in the social machine*
        
The error rate has dropped to about 10.4%.


In [205]:
""" Your code goes here. You need to implement the viterbi algorithm, and display the output that your code generates
    You also need to display the error rate for your algorithm.
    https://www.audiolabs-erlangen.de/resources/MIR/FMP/C5/C5S3_Viterbi.html 
"""
from ctypes import sizeof


train_df = pd.read_csv("./data/typos20Test.data", sep=' ', header = None, names=["State", "Output"])
test_df = pd.read_csv("./data/typos20Test.data", sep=' ', header = None, names=["State", "Output"])

"""
    Inputs:
        O: Observation Space O = {O1, ... On }
        S: State Space S = {S1, ... Sn}
        Pi: Initital Probabilities
        Y: Sequence of Observations
        A: Transition Matrix K x K
        B: Emission Matrix K x N
"""

def Viterbi(O, S, pi, Tm, Em, HMM):
    
    
    S_num = Tm.shape[0] #Possible states
    O_num = O.shape[0] #Observation Sequence Length
    #Log probabilities
    Tn_log = np.log(Tm.to_numpy()) #A Transition
    Em_log = np.log(Em.to_numpy()) #B Emission
    Pi_log = np.log(pi.to_numpy()) #C Initital

    #Initialize backtracking
    #And Optimal State matrix
    E = np.zeros((S_num, O_num-1)).astype(np.int32)
    D = np.zeros((S_num, O_num)) #Possible states I x N length(O)
    state_output = np.column_stack((O, S))
   # D_log = pd.Series(Counter(map(tuple, state_output))).unstack().fillna(0)
    
   # print(Tm_log.iloc[:, 10])
#   Returns:
#         S_opt (np.ndarray): Optimal state sequence of length N
#         D_log (np.ndarray): Accumulated log probability matrix
#         E (np.ndarray): Backtracking matrix
    
    #D[:, 0] = Em_log.add(Pi_log.values).iloc[:,0].values #Add the log probabilities using log(A*B) = log(A) + log(B)
    #display(Pi_log.shape)
    #display(Em_log[:,[0]].shape)
    #display(pd.DataFrame(Em_log))
    D[:, [0]] = Em_log[:,[0]] + Pi_log #Add the log probabilities using log(A*B) = log(A) + log(B)

    #print(Em_log)
    O = O.to_numpy()
    S = S.to_numpy()
    #display(np.round(Em_log, 0))
    display(Em_log.shape)
    
    for n in range(1, O_num):   #for all Outputs
        #print(O[n])
        for i in range(S_num):  #for all States 
            #print(i)
            temp_sum = Tn_log[:, i] + D[:, n - 1]
            D[i, n] = np.max(temp_sum) + Em_log[i, HMM.letters.index(O[n])]
            E[i, n-1] = np.argmax(temp_sum)

    #Backtracking
    S_opt = np.zeros(O_num).astype(np.int32)
    S_opt[-1] =  np.argmax(D[:, -1])
    for n in range(O_num - 2, -1, -1):
         S_opt[n] = E[int(S_opt[n + 1]), n]
    
    return S_opt, D, E
    
#def viterbi(HMM, ev):
train_test = HMM(test_df)
O = test_df['Output']
S = test_df['State']
#display(train_test.trans)
#print(train_test.trans)

S_opt, D, E = Viterbi(O, S, train_test.init, train_test.trans, train_test.em, train_test)

hidden_state = []
for i in S_opt:
    hidden_state.append(train_test.letters[i])
    

correct = 0
for i in range(len(S_opt)):
    if(train_test.letters[S_opt[i]] == S[i]):
        correct += 1


hidden_state
print("Optimal state sequence: ", ''.join(hidden_state[0:50]), "\n")
print("Actual state sequence:  ", ''.join(S.values[0:50]), "\n")
print("Accuracy of Optimal State Sequence: %", round((correct/len(S_opt)) * 100 ,4))
display(S_opt[0:20])



(27, 27)

Optimal state sequence:  _ntroduction_the_industrial_revolution_and_its_con 

Actual state sequence:   introduction_the_industrial_revolution_and_its_con 

Accuracy of Optimal State Sequence: % 89.8624


array([ 0, 14, 20, 18, 15,  4, 21,  3, 20,  9, 15, 14,  0, 20,  8,  5,  0,
        9, 14,  4], dtype=int32)

In [184]:
train_test.letters[S_opt[10]]
S[i]

'i'

In [154]:
display(train_test.em)
display(train_test.trans)
display(train_test.init)


Output,_,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
State,,,,,,,,,,,,,,,,,,,,,
_,0.992353,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294,...,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294
a,0.000787,0.789764,0.000787,0.000787,0.000787,0.000787,0.000787,0.000787,0.000787,0.000787,...,0.068504,0.000787,0.062205,0.000787,0.000787,0.000787,0.000787,0.000787,0.000787,0.061417
b,0.003984,0.003984,0.752988,0.003984,0.003984,0.003984,0.003984,0.023904,0.051793,0.003984,...,0.003984,0.003984,0.003984,0.003984,0.003984,0.031873,0.003984,0.003984,0.003984,0.003984
c,0.001639,0.001639,0.001639,0.757377,0.057377,0.001639,0.045902,0.001639,0.001639,0.001639,...,0.001639,0.001639,0.001639,0.001639,0.001639,0.047541,0.001639,0.055738,0.001639,0.001639
d,0.001942,0.001942,0.001942,0.040777,0.759223,0.046602,0.038835,0.001942,0.001942,0.001942,...,0.001942,0.001942,0.027184,0.001942,0.001942,0.001942,0.001942,0.046602,0.001942,0.001942
e,0.000466,0.000466,0.000466,0.000466,0.057369,0.805037,0.000466,0.000466,0.000466,0.000466,...,0.000466,0.062034,0.000466,0.000466,0.000466,0.000466,0.064832,0.000466,0.000466,0.000466
f,0.002000,0.002000,0.002000,0.022000,0.054000,0.002000,0.754000,0.020000,0.002000,0.002000,...,0.002000,0.040000,0.002000,0.044000,0.002000,0.026000,0.002000,0.002000,0.002000,0.002000
g,0.003067,0.003067,0.024540,0.003067,0.003067,0.003067,0.033742,0.748466,0.036810,0.003067,...,0.003067,0.003067,0.003067,0.042945,0.003067,0.049080,0.003067,0.003067,0.003067,0.003067
h,0.001292,0.001292,0.025840,0.001292,0.001292,0.001292,0.001292,0.041344,0.781654,0.001292,...,0.001292,0.001292,0.001292,0.001292,0.001292,0.001292,0.001292,0.001292,0.042636,0.001292


S',_,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
S,,,,,,,,,,,,,,,,,,,,,
_,0.000294,0.119118,0.045000,0.042647,0.030000,0.024706,0.030000,0.015588,0.043529,0.078529,...,0.000882,0.022647,0.070294,0.167059,0.008235,0.005588,0.051471,0.000294,0.000588,0.000294
a,0.060630,0.000787,0.024409,0.055118,0.025197,0.000787,0.007874,0.018898,0.001575,0.030709,...,0.000787,0.068504,0.071654,0.166929,0.014961,0.031496,0.002362,0.000787,0.024409,0.001575
b,0.007968,0.019920,0.003984,0.003984,0.003984,0.338645,0.003984,0.003984,0.003984,0.031873,...,0.003984,0.043825,0.011952,0.003984,0.131474,0.011952,0.003984,0.003984,0.083665,0.003984
c,0.047541,0.149180,0.001639,0.019672,0.001639,0.101639,0.001639,0.001639,0.100000,0.167213,...,0.001639,0.014754,0.006557,0.109836,0.042623,0.001639,0.001639,0.001639,0.008197,0.001639
d,0.440777,0.019417,0.001942,0.001942,0.017476,0.200000,0.001942,0.007767,0.001942,0.081553,...,0.001942,0.011650,0.029126,0.001942,0.048544,0.011650,0.003883,0.001942,0.007767,0.001942
e,0.318097,0.038713,0.004664,0.027985,0.047575,0.025187,0.046175,0.008862,0.004664,0.012127,...,0.003265,0.113340,0.084888,0.028918,0.000466,0.018190,0.001399,0.014459,0.022388,0.000466
f,0.356000,0.030000,0.002000,0.002000,0.002000,0.144000,0.036000,0.002000,0.002000,0.056000,...,0.002000,0.022000,0.006000,0.164000,0.030000,0.002000,0.002000,0.002000,0.016000,0.002000
g,0.266871,0.070552,0.003067,0.003067,0.003067,0.092025,0.003067,0.024540,0.079755,0.085890,...,0.003067,0.085890,0.085890,0.006135,0.027607,0.003067,0.003067,0.003067,0.027607,0.003067
h,0.072351,0.193798,0.001292,0.001292,0.001292,0.432817,0.001292,0.001292,0.001292,0.126615,...,0.001292,0.009044,0.003876,0.016796,0.016796,0.001292,0.001292,0.001292,0.009044,0.001292


,State
_,0.168112
a,0.061952
b,0.011164
c,0.029057
d,0.024322
e,0.105512
f,0.023575
g,0.014902
h,0.037231
i,0.068281


In [155]:
train_test.letters.index('a')

1

## Part III - Summarize your results
As part of this assignment, include a short report in your notebook describing your work. The report should contain the following:
### Purpose: 

This homework essentially uses a training and testing data set of typos. The training set can be thought of the HMM tables which are generated
knowing the correct 'state' and an incorrect 'output'. The purpose is to use the trained HMM model in conjunction with the dynamic programming Viterbi algorithm. Otherwise, this problem would be far too computationally difficult. The goal is to use the training set and reproduce the correct output from the test data. 

### Procedure:
3 Tables
##### Transition Probabilities

I created the transition table using a pandas dataframe. I simply shifted over the 'output' column by 1, to simulate the transition probability of going from the true state to the incorrect + 1 state. 

##### Emission Probabilities
Also using a pandas dataframe, I create an NxN matrix, where N is the count of unique characters. This was made by summing the probabilities of each individual character. 

##### Viterbi Algorithm

After creating the HMM with Pandas, I converted all my tables to numpy arrays for simplicity. 

In this section explain about the algorithm and the data structure you used for storing the values in the matrix with its back pointer.

### Data
Again, the first dataset typos20.data was used as a training set for the HMM mode. Whereas the typos20Test.data output column was only given to the Viterbi algorithm. The typos20Test.data state column was used to compare to the optimal state sequence output by Viterbi.

Explain about the data you used, including the data included in the data set and any pre-processing you did to the data.
### Results


Explain your observations, including the sentences that you tested and their corresponding outputs. Were there any sentences where your algorithm did not work?


Your report goes here.
